In [134]:
import pandas as pd 
import numpy as np
import pyblp
from sklearn.linear_model import LinearRegression

In [148]:

def run_problem_save_estimates(df, results, i):

    df = df[(df['shares'] >= 0.000001)
                          & (df['shares'] <= 1)]

    product_data = df
    # Setting up the formulations
    product_formulations = (
    pyblp.Formulation('1  + prices+ characteristic1 + characteristic2'),
    pyblp.Formulation('0 +  prices'),
    pyblp.Formulation('labor + capital')
    )
    product_formulations
    mc_integration = pyblp.Integration('monte_carlo', size=100, specification_options={'seed': 0})
    problem = pyblp.Problem(product_formulations, product_data, integration=mc_integration)
    bfgs = pyblp.Optimization('bfgs', {'gtol': 1e-6})
    estimation = problem.solve(sigma=(np.ones((1, 1))/2),beta=[2., -1.5, -0.5, -0.5], optimization=bfgs)
    estimation_beta = estimation.beta
    estimation_sigma = estimation.sigma
    estimation_results = np.reshpae(np.flatten(np.concatenate((estimation_beta, estimation_sigma))), (1,5))

    print




In [149]:
N_simulation = 1000

results = np.zeros((3, 5))
print(results)


for i in range(1, 3):
    file_path = f"../data/market_integrates_{i}.csv"
    df = pd.read_csv(file_path)
    run_problem_save_estimates(df, results, i-1)


print(results)


[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
Initializing the problem ...
Initialized the problem after 00:00:00.

Dimensions:
 T    N     F     I     K1    K2    K3    MD    MS 
---  ----  ---  -----  ----  ----  ----  ----  ----
100  1000  10   10000   4     1     3     3     3  

Formulations:
        Column Indices:            0       1            2                3       
-------------------------------  ------  ------  ---------------  ---------------
  X1: Linear Characteristics       1     prices  characteristic1  characteristic2
 X2: Nonlinear Characteristics   prices                                          
X3: Linear Cost Characteristics    1     labor       capital                     
Solving the problem ...

Nonlinear Coefficient Initial Values:
Sigma:     prices    
------  -------------
prices  +5.000000E-01

Beta Initial Values:
      1           prices      characteristic1  characteristic2
-------------  -------------  ---------------  ---------------
+2.00